In [2]:
import pandas as pd
import numpy as np
import xlsxwriter as xl
import re
import itertools
from datetime import datetime
pd.options.mode.chained_assignment = None



## TODO create a front end interface to select which tool and contest date
tool = "DK"
# tool = "Yahoo"
# sport = "NHL"
sport = "NBA"
contest_date = "jan25"

path = f"csv/{tool}_{sport}/{contest_date}.csv"


def print_current_df(dataframe):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(dataframe)

In [18]:
def run_script(tool, sport, date):
    path = f"csv/{tool}_{sport}/{contest_date}.csv"
    original_df = extract_from_csv(path)
    df = create_df(sport,tool,original_df)
    player_df = choose_relevant_players(df)
    output_sheet(player_df,tool,sport,contest_date)
run_script(tool,sport,contest_date)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,Team,FPPG,Game,Date,Time,Zone
0,C,Nikola Jokic (21122383),Nikola Jokic,21122383,C/UTIL,12500,DEN@DET 01/25/2022 07:00PM ET,DEN,60.14,DEN@DET,01/25/2022,07:00PM,ET
1,PG,James Harden (21122385),James Harden,21122385,PG/G/UTIL,11700,LAL@BKN 01/25/2022 07:30PM ET,BKN,52.19,LAL@BKN,01/25/2022,07:30PM,ET
2,C,Joel Embiid (21122388),Joel Embiid,21122388,C/UTIL,11300,NOP@PHI 01/25/2022 07:00PM ET,PHI,53.24,NOP@PHI,01/25/2022,07:00PM,ET
3,PG/SF,LeBron James (21122390),LeBron James,21122390,PG/SF/F/G/UTIL,11100,LAL@BKN 01/25/2022 07:30PM ET,LAL,54.05,LAL@BKN,01/25/2022,07:30PM,ET
4,PG,Luka Doncic (21122395),Luka Doncic,21122395,PG/G/UTIL,10900,DAL@GSW 01/25/2022 10:00PM ET,DAL,53.32,DAL@GSW,01/25/2022,10:00PM,ET
5,SF,Kevin Durant (21122398),Kevin Durant,21122398,SF/F/UTIL,10800,LAL@BKN 01/25/2022 07:30PM ET,BKN,50.66,LAL@BKN,01/25/2022,07:30PM,ET
6,PG,Stephen Curry (21122401),Stephen Curry,21122401,PG/G/UTIL,10700,DAL@GSW 01/25/2022 10:00PM ET,GSW,46.88,DAL@GSW,01/25/2022,10:00PM,ET
7,SF/PF,Jayson Tatum (21122404),Jayson Tatum,21122404,SF/PF/F/UTIL,10400,SAC@BOS 01/25/2022 07:30PM ET,BOS,46.11,SAC@BOS,01/25/2022,07:30PM,ET
8,PF,Paul George (21122408),Paul George,21122408,PF/F/UTIL,10300,LAC@WAS 01/25/2022 07:00PM ET,LAC,46.18,LAC@WAS,01/25/2022,07:00PM,ET
9,PF,Kawhi Leonard (21122411),Kawhi Leonard,21122411,PF/F/UTIL,10200,LAC@WAS 01/25/2022 07:00PM ET,LAC,0.00,LAC@WAS,01/25/2022,07:00PM,ET


Name Of Player:all


,Position,Name,Team,Game,Time,FPPG,Salary
0,C,Nikola Jokic,DEN,DEN@DET,07:00PM,60.14,12500
1,PG,James Harden,BKN,LAL@BKN,07:30PM,52.19,11700
2,C,Joel Embiid,PHI,NOP@PHI,07:00PM,53.24,11300
3,PG/SF,LeBron James,LAL,LAL@BKN,07:30PM,54.05,11100
4,PG,Luka Doncic,DAL,DAL@GSW,10:00PM,53.32,10900
5,SF,Kevin Durant,BKN,LAL@BKN,07:30PM,50.66,10800
6,PG,Stephen Curry,GSW,DAL@GSW,10:00PM,46.88,10700
7,SF/PF,Jayson Tatum,BOS,SAC@BOS,07:30PM,46.11,10400
8,PF,Paul George,LAC,LAC@WAS,07:00PM,46.18,10300
9,PF,Kawhi Leonard,LAC,LAC@WAS,07:00PM,0.00,10200


Name Of Player:exit


,Position,Name,Team,Game,Time,FPPG,Salary
0,C,Nikola Jokic,DEN,DEN@DET,07:00PM,60.14,12500
1,PG,James Harden,BKN,LAL@BKN,07:30PM,52.19,11700
2,C,Joel Embiid,PHI,NOP@PHI,07:00PM,53.24,11300
3,PG/SF,LeBron James,LAL,LAL@BKN,07:30PM,54.05,11100
4,PG,Luka Doncic,DAL,DAL@GSW,10:00PM,53.32,10900
5,SF,Kevin Durant,BKN,LAL@BKN,07:30PM,50.66,10800
6,PG,Stephen Curry,GSW,DAL@GSW,10:00PM,46.88,10700
7,SF/PF,Jayson Tatum,BOS,SAC@BOS,07:30PM,46.11,10400
8,PF,Paul George,LAC,LAC@WAS,07:00PM,46.18,10300
9,PF,Kawhi Leonard,LAC,LAC@WAS,07:00PM,0.00,10200


succesfully created: output/DK/NBA/jan25_DK_NBA.xlsx


In [3]:
from IPython.display import display
import ipywidgets as widgets


def sport_selected(arg):
    print("button has been clicked!", arg.description)
    sport = arg.description
    select_tool()

def select_tool():
    button_dk = widgets.Button(description = 'DK')   
    button_yahoo = widgets.Button(description = 'Yahoo')   
    button_dk.on_click(tool_selected)
    button_yahoo.on_click(tool_selected)
    display(button_dk)
    display(button_yahoo)
    
    
def tool_selected(arg):
    print("button has been clicked!", arg.description)
    sport = arg.description
    
    
    
# button_nhl = widgets.Button(description = 'NHL')   
# button_nba = widgets.Button(description = 'NBA')   
# button_nhl.on_click(sport_selected)
# button_nba.on_click(sport_selected)
# display(button_nhl)
# display(button_nba)
# widgets.DatePicker(
#     description='Pick a Date',
#     disabled=False
# )

In [4]:
def extract_from_csv(path):
    original_df = pd.read_csv(path)
    if tool == "DK":
        original_df[['Game','Date','Time','Zone']] = original_df.loc[:,'Game Info'].str.split(" ", 3, expand = True)
        original_df = original_df.rename(columns={'AvgPointsPerGame':'FPPG','TeamAbbrev':'Team'})
    
    print_current_df(original_df)
    return original_df
original_df = extract_from_csv(path)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,Team,FPPG,Game,Date,Time,Zone
0,C,Nikola Jokic (21122383),Nikola Jokic,21122383,C/UTIL,12500,DEN@DET 01/25/2022 07:00PM ET,DEN,60.14,DEN@DET,01/25/2022,07:00PM,ET
1,PG,James Harden (21122385),James Harden,21122385,PG/G/UTIL,11700,LAL@BKN 01/25/2022 07:30PM ET,BKN,52.19,LAL@BKN,01/25/2022,07:30PM,ET
2,C,Joel Embiid (21122388),Joel Embiid,21122388,C/UTIL,11300,NOP@PHI 01/25/2022 07:00PM ET,PHI,53.24,NOP@PHI,01/25/2022,07:00PM,ET
3,PG/SF,LeBron James (21122390),LeBron James,21122390,PG/SF/F/G/UTIL,11100,LAL@BKN 01/25/2022 07:30PM ET,LAL,54.05,LAL@BKN,01/25/2022,07:30PM,ET
4,PG,Luka Doncic (21122395),Luka Doncic,21122395,PG/G/UTIL,10900,DAL@GSW 01/25/2022 10:00PM ET,DAL,53.32,DAL@GSW,01/25/2022,10:00PM,ET
5,SF,Kevin Durant (21122398),Kevin Durant,21122398,SF/F/UTIL,10800,LAL@BKN 01/25/2022 07:30PM ET,BKN,50.66,LAL@BKN,01/25/2022,07:30PM,ET
6,PG,Stephen Curry (21122401),Stephen Curry,21122401,PG/G/UTIL,10700,DAL@GSW 01/25/2022 10:00PM ET,GSW,46.88,DAL@GSW,01/25/2022,10:00PM,ET
7,SF/PF,Jayson Tatum (21122404),Jayson Tatum,21122404,SF/PF/F/UTIL,10400,SAC@BOS 01/25/2022 07:30PM ET,BOS,46.11,SAC@BOS,01/25/2022,07:30PM,ET
8,PF,Paul George (21122408),Paul George,21122408,PF/F/UTIL,10300,LAC@WAS 01/25/2022 07:00PM ET,LAC,46.18,LAC@WAS,01/25/2022,07:00PM,ET
9,PF,Kawhi Leonard (21122411),Kawhi Leonard,21122411,PF/F/UTIL,10200,LAC@WAS 01/25/2022 07:00PM ET,LAC,0.00,LAC@WAS,01/25/2022,07:00PM,ET


### Modify Original CSV entries to create a DF of relevant data


In [13]:
def convert_positions_to_values(df,original_df):
    
    df.insert(loc=1, column='Pos', value=0)
    conditions = [
        (df['Position'] == 'PG'),
        (df['Position'] == 'SG'),
        (df['Position'] == 'SF'),
        (df['Position'] == 'PF'),
        (df['Position'] == 'C')
    ]
    values = [1, 2, 3, 4, 5]
    df.loc[:,'Pos'] = np.select(conditions, values)

    
def single_game(df,original_df):
    df = original_df[['FPPG','Salary']]
    df.insert(0, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
    
def drop_inactive_players(df):
    df = df[df['Injury Status'] != 'INJ'] 
    df = df[df['Injury Status'] != 'O'] 
    df = df[df['Injury Status'] != 'IR'] 
    return df
def create_df(sport,tool,original_df):
    df = pd.DataFrame()
    if tool == "Yahoo":
        df = original_df[['Position', 'Team','Game','Time','Injury Status','FPPG','Salary']].copy()
        df.insert(4, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
        df = drop_inactive_players(df)
        
        if sport == "NHL":
            df.insert(df.columns.get_loc("FPPG"),'Starting',original_df['Starting'])
            df['Value'] = df['FPPG']/df['Salary']
        else:
            number_of_teams = len(original_df['Team'].value_counts())
            if(number_of_teams == 2):
                single_game(df,original_df)
            else:    
                convert_positions_to_values(df,original_df)
                df.head()
    if tool == "DK":
        df = original_df[['Position', 'Name','Team','Game','Time','FPPG','Salary']]
        if sport == "NHL":
            df['Value'] = df['FPPG']/df['Salary']*1000
            df.insert(df.columns.get_loc("FPPG"),'Starting','')
            df.loc[df.Position == 'G','Starting'] = 'No'
    if sport == "NHL":
        df = df.round({'Value': 2})
        df.loc[df['Position'] == 'LW', 'Position'] = 'W'
        df.loc[df['Position'] == 'RW', 'Position'] = 'W'
#     print_current_df(df)
    return df
    
df = create_df(sport,tool,original_df)

### Interface

To be modified as more front end functionality is added

In [6]:
def add_to_lineup(df, player_df, name):
    if player_df['Name'].str.lower().str.contains(name.lower()).any() != True:
        player_df = player_df.append(df.loc[df['Name'].str.lower() == name.lower()])
    else:
        if name != '' : print(name + " is already in list")
    return player_df
def remove_from_lineup(player_df, name):
    print("name? " + name)
    if player_df['Name'].str.lower().str.contains(name.lower()).any() == True:
        player_df.drop(player_df.loc[player_df['Name'].str.lower()==name.lower()].index, inplace=True)
    else:
        if name != ''  : print(name + " is not in in list") 
    return player_df

In [7]:
df2 = df.sort_values(by=['Salary'], ascending = False)
# if sport == 'NHL':
#     df2 = df2.drop(df2[df2.Starting == "No"].index)
df2.drop(df2[df2.FPPG == 0.00].index,inplace=True)
for name in df2['Name']: 
    print(name, end = ", ")


Nikola Jokic, James Harden, Joel Embiid, LeBron James, Luka Doncic, Kevin Durant, Stephen Curry, Jayson Tatum, Paul George, Dejounte Murray, Anthony Davis, Damian Lillard, Karl-Anthony Towns, Kyrie Irving, Jaylen Brown, Fred VanVleet, Bradley Beal, Pascal Siakam, LaMelo Ball, Jusuf Nurkic, Miles Bridges, Russell Westbrook, Brandon Ingram, Kyle Kuzma, Christian Wood, Jonas Valanciunas, Terry Rozier, D'Angelo Russell, Anthony Edwards, Kristaps Porzingis, CJ McCollum, Tyrese Haliburton, Tobias Harris, De'Aaron Fox, Cade Cunningham, Anfernee Simons, Robert Williams, Jerami Grant, Draymond Green, OG Anunoby, Norman Powell, Spencer Dinwiddie, Dennis Schroder, Aaron Gordon, Reggie Jackson, Scottie Barnes, Josh Hart, Marcus Morris Sr., Tyrese Maxey, Saddiq Bey, Jakob Poeltl, Kevin Porter Jr., Harrison Barnes, Gordon Hayward, Buddy Hield, Jalen Brunson, Andrew Wiggins, Terence Davis, Will Barton, Derrick White, Marcus Smart, Al Horford, Seth Curry, Chris Boucher, Hamidou Diallo, Jordan Poole, N

### Table Add

In [8]:
def get_names(names):
    return [x.strip() for x in names.split(',')]


##adding these blank columns that can be edited in an excel sheet to add the values
##Future versions will let user change the values inside them
##TODO Modify current excel based version to use excel formulas
##https://stackoverflow.com/questions/51348874/save-pandas-dataframes-with-formulas-to-xlsx-files
def add_extra_columns(player_df):
    player_df['Floor'] = ""
    player_df['Ceiling'] = ""
    if sport == 'NBA':
        player_df['Estimate'] = ""
    player_df['Actual'] = ""
    if sport == 'NBA':
        player_df['Deviation'] = ""
    player_df['Floor Value'] = ""
    player_df['Ceiling Value'] = ""
    player_df['Border Value'] = ""
    
    if sport == 'NBA':
        player_df['Estimate Value'] = ""
        player_df['Actual Value'] = ""
    player_df['Notes'] = ""
    return player_df
def choose_relevant_players(df):
    player_df = pd.DataFrame(columns = df.columns)
    queries = ['exit', 'all', 'show']
    player_df = player_df.replace(np.nan, '', regex=True)
    
    exit = False
    while exit!=True:
        query = input('Name Of Player:')
        if query not in queries:
            if re.search('^remove', query):
                names_to_remove = re.findall('(?<=remove).*$', query)[0]
                names = get_names(names_to_remove)
                for name in names:
                    name = name.title()
                    player_df = remove_from_lineup(player_df,name)
                    print(f"removed {name}")

            else:
                names = get_names(query)
                for name in names:
                    name = name.title()
                    if df['Name'].str.lower().str.contains(name.lower()).any():
                        player_df = add_to_lineup(df,player_df,name)   
                    else:
                        print(name + " is not in the csv list. Please try again")
        elif query =='show':
            print_current_df(player_df)
        elif query =='all':
            player_df = df
            print_current_df(player_df)
        else:
            print_current_df(player_df)
            player_df = add_extra_columns(player_df)
            exit = True
    return player_df


player_df = choose_relevant_players(df)



Name Of Player:
Name Of Player:all


,Position,Name,Team,Game,Time,FPPG,Salary
0,C,Nikola Jokic,DEN,DEN@DET,07:00PM,60.14,12500
1,PG,James Harden,BKN,LAL@BKN,07:30PM,52.19,11700
2,C,Joel Embiid,PHI,NOP@PHI,07:00PM,53.24,11300
3,PG/SF,LeBron James,LAL,LAL@BKN,07:30PM,54.05,11100
4,PG,Luka Doncic,DAL,DAL@GSW,10:00PM,53.32,10900
5,SF,Kevin Durant,BKN,LAL@BKN,07:30PM,50.66,10800
6,PG,Stephen Curry,GSW,DAL@GSW,10:00PM,46.88,10700
7,SF/PF,Jayson Tatum,BOS,SAC@BOS,07:30PM,46.11,10400
8,PF,Paul George,LAC,LAC@WAS,07:00PM,46.18,10300
9,PF,Kawhi Leonard,LAC,LAC@WAS,07:00PM,0.00,10200


Name Of Player:exit


,Position,Name,Team,Game,Time,FPPG,Salary
0,C,Nikola Jokic,DEN,DEN@DET,07:00PM,60.14,12500
1,PG,James Harden,BKN,LAL@BKN,07:30PM,52.19,11700
2,C,Joel Embiid,PHI,NOP@PHI,07:00PM,53.24,11300
3,PG/SF,LeBron James,LAL,LAL@BKN,07:30PM,54.05,11100
4,PG,Luka Doncic,DAL,DAL@GSW,10:00PM,53.32,10900
5,SF,Kevin Durant,BKN,LAL@BKN,07:30PM,50.66,10800
6,PG,Stephen Curry,GSW,DAL@GSW,10:00PM,46.88,10700
7,SF/PF,Jayson Tatum,BOS,SAC@BOS,07:30PM,46.11,10400
8,PF,Paul George,LAC,LAC@WAS,07:00PM,46.18,10300
9,PF,Kawhi Leonard,LAC,LAC@WAS,07:00PM,0.00,10200


In [16]:
def output_sheet(output_df,tool,sport,contest_date):
#     dt = datetime.now()
#     now = dt.strftime("%Y-%m-%d %H_%M")

    file_name = f'output/{tool}/{sport}/{contest_date}_{tool}_{sport}.xlsx'
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
    # header and index and skip one row to allow us to insert a user defined
    # header.

    output_df.to_excel(writer, index = False, header=True)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Get the dimensions of the dataframe.
    (max_row, max_col) = output_df.shape

    # Create a list of column headers, to use in add_table().exo
    column_settings = []
    for header in player_df.columns:
        column_settings.append({'header': header})

    # Add the table.
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

    # Make the columns wider for clarity.
    worksheet.set_column(0, max_col - 1, 12)

    if sport == 'NHL': 
        superstar_df = df.loc[df['Salary'] > 18]
        superstar_df.to_excel (writer, index = False, header=True, sheet_name = "Superstars")
        worksheet2 = writer.sheets['Superstars']

        (max_row, max_col) = df.shape
        print_current_df(superstar_df)
        worksheet2.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

        # Make the columns wider for clarity.
        worksheet2.set_column(0, max_col - 1, 12)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()
    print('succesfully created: ' + file_name)

    
output_sheet(player_df,tool,sport,contest_date)


succesfully created: output/DK/NBA/jan25_DK_NBA.xlsx


### Single-game analysis [WIP]

#### Single Game Entry Strategy:

- **Stars:** Out of the 4-5 likely star players, choose 3.

- **Peasants:** Out of the 5-10 non-star players, choose 5.



In [ ]:
single_df = player_df[['Name','Salary','Floor','Ceiling','Estimate']]
print_current_df(single_df)
create_lineup(single_df)

In [ ]:
max_salary = 30

In [ ]:
## best 5 probable, best 5 worst, best 5 best. Create combinations of lineups with the most recurring names

def create_lineup(df):
    lineup = []
    df2 = df.ge()
        
    
    print(highest_paid_player.Name)
    

In [ ]:
numbers = [1, 2, 3, 7, 7, 9, 10]
result = [seq for i in range(len(numbers), 0, -1) for seq in itertools.combinations(numbers, i) if sum(seq) >= 10]
print(result)

In [ ]:
class Lineup: 
    superstar = ""
    megastar = ""
    star = ""
    other_players = []   
    

In [ ]:
class MyClass:
    """A simple example class"""
    i = 12345

    def __init__(self,data):
        self.data = data
l = [1,2,"e"]
x = MyClass(l)
x.data


### Multi-game analysis [WIP]

In [ ]:
# sorted_df = filled_df.sort_values(by=['Pos','Probable'],ascending=[True,False]).drop(['Notes', 'Pts', 'Reb', 'Ast', 'Stl', 'Blk', 'TO',
#        'Total'],axis = 1)

# # sorted_df.insert(12,'Worst Value',sorted_df["Worst"] / sorted_df["Salary"])
# # sorted_df.insert(13,'Best Value',sorted_df["Best"] / sorted_df["Salary"])
# df=sorted_df
# lineups = []
# positions = []



# df = df.loc[:, lambda df: ['Pos','Salary','Name', "Worst","Best",'Probable']]
# [positions.append(df.loc[(sorted_df['Pos'] == i)].sort_values(by=['Pos','Probable'],ascending=[True,False])) for i in range(1,6)]

# [print_current_df(position_df) for position_df in positions]

In [ ]:
# def select_eligible(df):
# new_df = sorted_df.loc[(sorted_df['Salary'] > 30)]
# df

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
